In [1]:
from __future__ import print_function, division
import requests
import pandas as pd
import numpy as np
import os.path
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("darkgrid")

%matplotlib inline

from sklearn.cross_validation import train_test_split

from sklearn.linear_model import LinearRegression, Lasso, LassoCV
import statsmodels.formula.api as smf

/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


# Project Luther - Regressing 2017-2018 NBA player salaries on 2016-2017 player statistics

**Data for 2016-2017 player statistics** - using Beautiful Soup

In [2]:
url = 'https://www.basketball-reference.com/pi/shareit/spFhr'
response = requests.get(url)
page = response.text

from bs4 import BeautifulSoup
soup = BeautifulSoup(page,"lxml")

In [3]:
#This takes the player stats data, and creates a list of a lists, where a list is all the values of a player
results = []
for row in soup.find_all('tr')[1:]:
    data = row.find_all('td')
    Name = data[0].text
    Pos = data[1].text
    Age = data[2].text
    Tm = data[3].text
    G = data[4].text
    GS = data[5].text
    MP = data[6].text
    FG = data[7].text
    FGA = data[8].text
    FG_Per = data[9].text
    threeP = data[10].text
    threePA = data[11].text
    threeP_Perc = data[12].text
    twoP = data[13].text
    twoPA = data[14].text
    twoP_Per = data[15].text
    eFG_Per = data[16].text
    FT = data[17].text
    FTA = data[18].text
    FT_Per = data[19].text
    ORB = data[20].text
    DRB = data[21].text
    TRB = data[22].text
    AST = data[23].text
    STL = data[24].text
    BLK = data[25].text
    TOV = data[26].text
    PF = data[27].text
    PS_Game = data[28].text
    # get the rest of the data you need about each coin here, then add it to the dictionary that you append to results
    results.append([Name, Pos, Age, Tm, G, GS, MP, FG, FGA, FG_Per, threeP, threePA, threeP_Perc, twoP, twoPA, twoP_Per, eFG_Per, FT, FTA, FT_Per, ORB, DRB, TRB, AST, STL, BLK, TOV, PF, PS_Game])

In [4]:
#create a pandas dataframe
pd.set_option('display.max_columns', 500)
players_df = pd.DataFrame(results, columns=['name', 'Pos', 'Age', 'Tm', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG_Per', 'threeP', 'threePA', 'threeP_Perc', 'twoP', 'twoPA', 'twoP_Per', 'eFG_Per', 'FT', 'FTA', 'FT_Per', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PS_Game'])

In [5]:
#replace missing values with NaN
players_df.replace(r'^\s*$', np.nan, regex=True, inplace=True)
#replace NaN values with 0 
players_df.fillna(value=0, inplace=True)

In [6]:
#sanity check
players_df.head(1)

,name,Pos,Age,Tm,G,GS,MP,FG,FGA,FG_Per,threeP,threePA,threeP_Perc,twoP,twoPA,twoP_Per,eFG_Per,FT,FTA,FT_Per,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PS_Game
0,Alex Abrines,SG,23,OKC,68,6,15.5,2.0,5.0,.393,1.4,3.6,.381,0.6,1.4,.426,.531,0.6,0.7,.898,0.3,1.0,1.3,0.6,0.5,0.1,0.5,1.7,6.0


**Data for 2016-2017 player salaries** - using Beautiful Soup

In [7]:
salaries_url = 'https://www.basketball-reference.com/pi/shareit/XDz3w'
salaries_response = requests.get(salaries_url)
page = salaries_response.text

from bs4 import BeautifulSoup
soup = BeautifulSoup(page,"lxml")

In [8]:
#This takes the player salaries data, and creates a list of a lists, where a list is all the values of a player
salaries = []
for x in soup.find_all('tr')[2:]:
    tds_salaries = x.find_all('td')
    name_s = tds_salaries[0].text
    salary = tds_salaries[2].text
    salaries.append([name_s, salary[1:]])

In [9]:
#create a salary pandas dataframe
salaries_df = pd.DataFrame(salaries, columns=['name', 'salary'])

In [10]:
#take out all the commas in salary column
salaries_df['salary'] = salaries_df['salary'].str.replace(',', '')

In [11]:
#sanity check
salaries_df[0:1]

,name,salary
0,Stephen Curry,34682550


In [12]:
#there are duplicate rows for people that got traded..so we need to groupby and calculate their average salary!
#convert salary column to integers - so I can take the mean salaries of the players that got traded-remove duplicates!!
salaries_df['salary'] = pd.to_numeric(salaries_df['salary'], errors='coerce', downcast='float').fillna(0)

In [13]:
#takes the mean salaries of players that got traded (removes duplicates)
groupbyName = salaries_df.groupby("name")
salary_df = groupbyName.salary.agg([np.mean]).reset_index()

# Merging Stats & Salaries together

In [14]:
#then merge the 2 dataframes on column "Name" - no one in the NBA with the same name
nba_df = pd.merge(players_df, salary_df, how='left', on='name')

In [15]:
#drop all the rows with NaN values for 'salary' -- either they retired or did not make a team this year
nba_df = nba_df.dropna()

In [16]:
nba_df.shape

(415, 30)

In [17]:
#sanity check
nba_df.isnull().values.any() 

False

**Data clean**

In [18]:
#let's clean up our data a little more 
#convert the numerial columns to floats
nba_df[['Age','G', 'GS', 'MP', 'FG', 'FGA', 'FG_Per', 'threeP', 'threePA', 'threeP_Perc', 'twoP', 'twoPA', 'twoP_Per', 'eFG_Per', 'FT', 'FTA', 'FT_Per', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PS_Game']] = nba_df[['Age','G', 'GS', 'MP', 'FG', 'FGA', 'FG_Per', 'threeP', 'threePA', 'threeP_Perc', 'twoP', 'twoPA', 'twoP_Per', 'eFG_Per', 'FT', 'FTA', 'FT_Per', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PS_Game']].apply(pd.to_numeric, errors='coerce')

In [19]:
#Rename 'mean' column to 'salary'
nba_df = nba_df.rename(columns = {'mean':'salary'})

# Add in a binary playoff column: 0 = no playoffs, 1 = playoffs

In [20]:
#for the teams that got traded, need to specify which team they got traded to
nba_df.loc[nba_df['name'] == 'Quincy Acy', "Tm"] = "BRK"
nba_df.loc[nba_df['name'] == 'Justin Anderson', "Tm"] = "PHI"
nba_df.loc[nba_df['name'] == 'Matt Barnes', "Tm"] = "GSW"
nba_df.loc[nba_df['name'] == 'Bojan Bogdanovic', "Tm"] = "WAS"
nba_df.loc[nba_df['name'] == 'Andrew Bogut', "Tm"] = "DAL"
nba_df.loc[nba_df['name'] == 'Corey Brewer', "Tm"] = "LAL"
nba_df.loc[nba_df['name'] == 'Jose Calderon', "Tm"] = "ATL"
nba_df.loc[nba_df['name'] == 'Omri Casspi', "Tm"] = "MIN"
nba_df.loc[nba_df['name'] == 'Quinn Cook', "Tm"] = "NOP"
nba_df.loc[nba_df['name'] == 'DeMarcus Cousins', "Tm"] = "NOP"
nba_df.loc[nba_df['name'] == 'Mike Dunleavy', "Tm"] = "ATL"
nba_df.loc[nba_df['name'] == 'Tyler Ennis', "Tm"] = "LAL"
nba_df.loc[nba_df['name'] == 'Tyreke Evans', "Tm"] = "SAC"
nba_df.loc[nba_df['name'] == 'Yogi Ferrell', "Tm"] = "DAL"
nba_df.loc[nba_df['name'] == 'Langston Galloway', "Tm"] = "SAC"
nba_df.loc[nba_df['name'] == 'Taj Gibson', "Tm"] = "OKC"
nba_df.loc[nba_df['name'] == 'Jerami Grant', "Tm"] = "OKC"
nba_df.loc[nba_df['name'] == 'Spencer Hawes', "Tm"] = "MIL"
nba_df.loc[nba_df['name'] == 'Buddy Hield', "Tm"] = "SAC"
nba_df.loc[nba_df['name'] == 'Serge Ibaka', "Tm"] = "TOR"
nba_df.loc[nba_df['name'] == 'Ersan Ilyasova', "Tm"] = "ATL"
nba_df.loc[nba_df['name'] == 'Brandon Jennings', "Tm"] = "WAS"
nba_df.loc[nba_df['name'] == 'Kyle Korver', "Tm"] = "CLE"
nba_df.loc[nba_df['name'] == 'Joffrey Lauvergne', "Tm"] = "CHI"
nba_df.loc[nba_df['name'] == 'DeAndre Liggins', "Tm"] = "HOU"
nba_df.loc[nba_df['name'] == 'Chris McCullough', "Tm"] = "WAS"
nba_df.loc[nba_df['name'] == 'K.J. McDaniels', "Tm"] = "BRK"
nba_df.loc[nba_df['name'] == 'Doug McDermott', "Tm"] = "OKC"
nba_df.loc[nba_df['name'] == 'Andrew Nicholson', "Tm"] = "BRK"
nba_df.loc[nba_df['name'] == 'Nerlens Noel', "Tm"] = "DAL"
nba_df.loc[nba_df['name'] == 'Jusuf Nurkic', "Tm"] = "POR"
nba_df.loc[nba_df['name'] == "Johnny O'Bryant", "Tm"] = "CHO"
nba_df.loc[nba_df['name'] == 'Cameron Payne', "Tm"] = "CHI"
nba_df.loc[nba_df['name'] == 'Mason Plumlee', "Tm"] = "DEN"
nba_df.loc[nba_df['name'] == 'Miles Plumlee', "Tm"] = "CHO"
nba_df.loc[nba_df['name'] == 'Chasson Randle', "Tm"] = "NYK"
nba_df.loc[nba_df['name'] == 'Terrence Ross', "Tm"] = "ORL"
nba_df.loc[nba_df['name'] == 'Wayne Selden', "Tm"] = "MEM"
nba_df.loc[nba_df['name'] == 'Lance Stephenson', "Tm"] = "IND"
nba_df.loc[nba_df['name'] == 'Axel Toupane', "Tm"] = "NOP"
nba_df.loc[nba_df['name'] == 'P.J. Tucker', "Tm"] = "TOR"
nba_df.loc[nba_df['name'] == 'Briante Weber', "Tm"] = "CHO"
nba_df.loc[nba_df['name'] == 'Deron Williams', "Tm"] = "CLE"
nba_df.loc[nba_df['name'] == 'Derrick Williams', "Tm"] = "CLE"
nba_df.loc[nba_df['name'] == 'Lou Williams', "Tm"] = "HOU"
nba_df.loc[nba_df['name'] == 'Troy Williams', "Tm"] = "HOU"

In [21]:
#create playoff column 
nba_df['playoff'] = 0 

In [22]:
#non-playoff teams = 0
#playoff teams: POR, LAC, OKC, MEM - CHI, ATL, MIL, IND, UTA, HOU - WAS, TOR, SAS, BOS, GSW, CLE
nba_df.loc[nba_df['Tm'] == 'POR', "playoff"] = 1
nba_df.loc[nba_df['Tm'] == 'LAC', "playoff"] = 1
nba_df.loc[nba_df['Tm'] == 'OKC', "playoff"] = 1
nba_df.loc[nba_df['Tm'] == 'MEM', "playoff"] = 1
nba_df.loc[nba_df['Tm'] == 'CHI', "playoff"] = 1
nba_df.loc[nba_df['Tm'] == 'ATL', "playoff"] = 1
nba_df.loc[nba_df['Tm'] == 'MIL', "playoff"] = 1
nba_df.loc[nba_df['Tm'] == 'IND', "playoff"] = 1
nba_df.loc[nba_df['Tm'] == 'UTA', "playoff"] = 1
nba_df.loc[nba_df['Tm'] == 'HOU', "playoff"] = 1
nba_df.loc[nba_df['Tm'] == 'WAS', "playoff"] = 1
nba_df.loc[nba_df['Tm'] == 'TOR', "playoff"] = 1
nba_df.loc[nba_df['Tm'] == 'SAS', "playoff"] = 1
nba_df.loc[nba_df['Tm'] == 'BOS', "playoff"] = 1
nba_df.loc[nba_df['Tm'] == 'GSW', "playoff"] = 1
nba_df.loc[nba_df['Tm'] == 'CLE', "playoff"] = 1

In [23]:
nba_df

,name,Pos,Age,Tm,G,GS,MP,FG,FGA,FG_Per,threeP,threePA,threeP_Perc,twoP,twoPA,twoP_Per,eFG_Per,FT,FTA,FT_Per,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PS_Game,salary,playoff
0,Alex Abrines,SG,23,OKC,68,6,15.5,2.0,5.0,0.393,1.4,3.6,0.381,0.6,1.4,0.426,0.531,0.6,0.7,0.898,0.3,1.0,1.3,0.6,0.5,0.1,0.5,1.7,6.0,5725000.0,1
1,Quincy Acy,PF,26,BRK,38,1,14.7,1.8,4.5,0.412,1.0,2.4,0.411,0.9,2.1,0.413,0.521,1.2,1.6,0.750,0.5,2.5,3.0,0.5,0.4,0.4,0.6,1.8,5.8,1709538.0,0
2,Steven Adams,C,23,OKC,80,80,29.9,4.7,8.2,0.571,0.0,0.0,0.000,4.7,8.2,0.572,0.571,2.0,3.2,0.611,3.5,4.2,7.7,1.1,1.1,1.0,1.8,2.4,11.3,22471910.0,1
3,Arron Afflalo,SG,31,SAC,61,45,25.9,3.0,6.9,0.440,1.0,2.5,0.411,2.0,4.4,0.457,0.514,1.4,1.5,0.892,0.1,1.9,2.0,1.3,0.3,0.1,0.7,1.7,8.4,1914326.0,0
4,Alexis Ajinca,C,28,NOP,39,15,15.0,2.3,4.6,0.500,0.0,0.1,0.000,2.3,4.5,0.511,0.500,0.7,1.0,0.725,1.2,3.4,4.5,0.3,0.5,0.6,0.8,2.0,5.3,4961798.0,0
5,Cole Aldrich,C,28,MIN,62,0,8.6,0.7,1.4,0.523,0.0,0.0,0.000,0.7,1.4,0.523,0.523,0.2,0.4,0.682,0.8,1.7,2.5,0.4,0.4,0.4,0.3,1.4,1.7,7300000.0,0
6,LaMarcus Aldridge,PF,31,SAS,72,72,32.4,6.9,14.6,0.477,0.3,0.8,0.411,6.6,13.8,0.480,0.488,3.1,3.8,0.812,2.4,4.9,7.3,1.9,0.6,1.2,1.4,2.2,17.3,21461010.0,1
8,Tony Allen,SG,35,MEM,71,66,27.0,3.9,8.4,0.461,0.2,0.8,0.278,3.6,7.6,0.479,0.473,1.1,1.8,0.615,2.3,3.2,5.5,1.4,1.6,0.4,1.4,2.5,9.1,2116955.0,1
9,Al-Farouq Aminu,PF,26,POR,61,25,29.1,3.0,7.6,0.393,1.1,3.5,0.330,1.9,4.2,0.445,0.468,1.6,2.2,0.706,1.3,6.1,7.4,1.6,1.0,0.7,1.5,1.7,8.7,7319035.0,1
12,Justin Anderson,SF,23,PHI,75,10,16.4,2.5,5.9,0.424,0.8,2.6,0.299,1.7,3.3,0.522,0.490,1.4,1.7,0.791,0.9,2.3,3.3,0.9,0.5,0.3,0.9,1.5,7.1,1579440.0,0


In [24]:
#save my dataset as a csv file 
nba_df.to_csv('nba_FINALDATA.csv', sep=',')